In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

file_path = "../../data/processed/medical_equipment_utilization_synthetic_cleaned(in).csv"

raw_df = pd.read_csv(file_path)
raw_df.columns = [c.strip().replace(" ", "_") for c in raw_df.columns]

df = raw_df.dropna(axis=1, how="all").copy()

currency_cols = [
    "Cost_per_procedure",
    "Daily_Operating_Cost",
    "Procedure_Revenue",
    "Net-Profit_(daily)",
]
for col in currency_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("KES", "", case=False, regex=False)
            .str.replace(",", "", regex=False)
            .str.replace(" ", "", regex=False)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

by_type = (
    df.groupby("Equipment_Type")
    .agg(
        total_revenue=("Procedure_Revenue", "sum"),
        total_operating_cost=("Daily_Operating_Cost", "sum"),
        total_net_profit=("Net-Profit_(daily)", "sum"),
        n_days=("Equipment_ID", "count"),
    )
)
by_type["roi_%"] = 100 * by_type["total_net_profit"] / by_type["total_operating_cost"]
by_type["profit_margin_%"] = 100 * by_type["total_net_profit"] / by_type["total_revenue"]
by_type["avg_profit_per_day"] = by_type["total_net_profit"] / by_type["n_days"]

by_dept = (
    df.groupby("Department")
    .agg(
        total_revenue=("Procedure_Revenue", "sum"),
        total_operating_cost=("Daily_Operating_Cost", "sum"),
        total_net_profit=("Net-Profit_(daily)", "sum"),
        n_days=("Equipment_ID", "count"),
    )
)
by_dept["roi_%"] = 100 * by_dept["total_net_profit"] / by_dept["total_operating_cost"]
by_dept["profit_margin_%"] = 100 * by_dept["total_net_profit"] / by_dept["total_revenue"]
by_dept["avg_profit_per_day"] = by_dept["total_net_profit"] / by_dept["n_days"]

by_type.head()


In [ ]:
fig_roi_type = px.bar(
    by_type.reset_index(),
    x="Equipment_Type",
    y="roi_%",
    hover_data=["profit_margin_%", "avg_profit_per_day"],
    title="ROI (%) by Equipment Type",
)
fig_roi_type.update_layout(xaxis_tickangle=-45)
fig_roi_type.show()


In [ ]:
fig_profit_day = px.bar(
    by_type.reset_index(),
    x="Equipment_Type",
    y="avg_profit_per_day",
    title="Average Profit per Day (KES) by Equipment Type",
)
fig_profit_day.update_layout(xaxis_tickangle=-45)
fig_profit_day.show()


In [ ]:
fig_roi_dept = px.bar(
    by_dept.reset_index(),
    x="Department",
    y="roi_%",
    hover_data=["profit_margin_%", "avg_profit_per_day"],
    title="ROI (%) by Department",
)
fig_roi_dept.update_layout(xaxis_tickangle=-30)
fig_roi_dept.show()

fig_profit_dept = px.bar(
    by_dept.reset_index(),
    x="Department",
    y="avg_profit_per_day",
    title="Average Profit per Day (KES) by Department",
)
fig_profit_dept.update_layout(xaxis_tickangle=-30)
fig_profit_dept.show()
